In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
import joblib

In [2]:
df = pd.read_csv('/heart-disease-UCI.csv')
print(df.describe)

<bound method NDFrame.describe of      age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   3       145   233    1        0      150      0      2.3   
1     37    1   2       130   250    0        1      187      0      3.5   
2     41    0   1       130   204    0        0      172      0      1.4   
3     56    1   1       120   236    0        1      178      0      0.8   
4     57    0   0       120   354    0        1      163      1      0.6   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   57    0   0       140   241    0        1      123      1      0.2   
299   45    1   3       110   264    0        1      132      0      1.2   
300   68    1   0       144   193    1        1      141      0      3.4   
301   57    1   0       130   131    0        1      115      1      1.2   
302   57    0   1       130   236    0        0      174      0      0.0   

     slope  ca  thal  target  
0        0   0     1  

In [3]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB
None


In [4]:
df.target.value_counts(normalize=True)

,proportion
target,
1,0.544554
0,0.455446


In [5]:

numeric_cols = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
                'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']


X = df[numeric_cols]
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=numeric_cols).reset_index(drop=True)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=numeric_cols).reset_index(drop=True)

print(X_train_scaled_df.head())


        age       sex        cp  trestbps      chol       fbs   restecg  \
0 -1.356798  0.722504  0.008099 -0.616856  0.914034 -0.383301  0.843133   
1  0.385086  0.722504 -0.971891  1.169491  0.439527 -0.383301 -1.046109   
2 -0.921327  0.722504  0.988089  1.169491 -0.300704 -0.383301  0.843133   
3  0.058483 -1.384075  0.008099  0.276318  0.059921 -0.383301 -1.046109   
4  0.602822  0.722504 -0.971891 -0.795490 -0.319684  2.608918  0.843133   

    thalach     exang   oldpeak     slope        ca      thal  
0  0.532781 -0.676632 -0.920864  0.953905 -0.689701 -0.509048  
1 -1.753582  1.477907 -0.193787  0.953905 -0.689701  1.178480  
2 -0.139679 -0.676632  2.350982 -0.694988 -0.689701 -0.509048  
3  0.487950 -0.676632  0.351521 -0.694988 -0.689701 -0.509048  
4  0.443119  1.477907  0.351521  0.953905  1.333421  1.178480  


In [6]:
train_final = pd.concat([X_train_scaled_df, y_train.reset_index(drop=True)], axis=1)
test_final = pd.concat([X_test_scaled_df, y_test.reset_index(drop=True)], axis=1)

X_train = train_final.drop('target', axis=1)
y_train = train_final['target']

X_test = test_final.drop('target', axis=1)
y_test = test_final['target']


In [23]:
def cross_validate_model(model, X, y, cv=5, model_name="Model"):
    print(f"\n🔁 Cross-Validation for {model_name.upper()} (cv={cv})")
    scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
    print(f"✅ Accuracy Scores: {scores}")
    print(f"📊 Mean Accuracy: {scores.mean():.4f}")
    print(f"📉 Standard Deviation: {scores.std():.4f}")

def accuracy(y_test,y_pred):
  print("✅ Accuracy:", accuracy_score(y_test, y_pred))

def confusionMatrix(y_test,y_pred):
  print("📊 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

def classificationReport(y_test,y_pred):
  print("📝 Classification Report:\n", classification_report(y_test, y_pred))


In [22]:
model = SVC(kernel='poly', random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("\n🔷 SVM ------------------")
print("✅ Accuracy:", round(accuracy_score(y_test, y_pred), 4))

confusionMatrix(y_test,y_pred)
classificationReport(y_test,y_pred)
cross_validate_model(model,X_train,y_train,model_name="SVM (poly)")


🔷 SVM ------------------
✅ Accuracy: 0.9016
📊 Confusion Matrix:
 [[26  3]
 [ 3 29]]
📝 Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.90      0.90        29
           1       0.91      0.91      0.91        32

    accuracy                           0.90        61
   macro avg       0.90      0.90      0.90        61
weighted avg       0.90      0.90      0.90        61


🔁 Cross-Validation for SVM (POLY) (cv=5)
✅ Accuracy Scores: [0.75510204 0.7755102  0.75       0.77083333 0.79166667]
📊 Mean Accuracy: 0.7686
📉 Standard Deviation: 0.0149


In [24]:
log_model = LogisticRegression(random_state=42,max_iter=100)
log_model.fit(X_train,y_train)
y_pred_log = log_model.predict(X_test)

print("\n🔷 Logistic Regression ------------------")
accuracy(y_test,y_pred_log)
confusionMatrix(y_test,y_pred_log)
classificationReport(y_test,y_pred_log)
cross_validate_model(model,X_train,y_train,model_name="LogisticRegression")


🔷 Logistic Regression ------------------
✅ Accuracy: 0.8524590163934426
📊 Confusion Matrix:
 [[25  4]
 [ 5 27]]
📝 Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.86      0.85        29
           1       0.87      0.84      0.86        32

    accuracy                           0.85        61
   macro avg       0.85      0.85      0.85        61
weighted avg       0.85      0.85      0.85        61


🔁 Cross-Validation for LOGISTICREGRESSION (cv=5)
✅ Accuracy Scores: [0.75510204 0.7755102  0.75       0.77083333 0.79166667]
📊 Mean Accuracy: 0.7686
📉 Standard Deviation: 0.0149


In [25]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform', metric='euclidean')
knn.fit(X_train,y_train)
y_pred_knn = knn.predict(X_test)

print("\n🔷 K-Nearest Neighbors ------------------")
accuracy(y_test,y_pred_knn)
confusionMatrix(y_test,y_pred_knn)
classificationReport(y_test,y_pred_knn)
cross_validate_model(model,X_train,y_train,model_name="K-Nearest Neighbors")


🔷 K-Nearest Neighbors ------------------
✅ Accuracy: 0.9016393442622951
📊 Confusion Matrix:
 [[27  2]
 [ 4 28]]
📝 Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.93      0.90        29
           1       0.93      0.88      0.90        32

    accuracy                           0.90        61
   macro avg       0.90      0.90      0.90        61
weighted avg       0.90      0.90      0.90        61


🔁 Cross-Validation for K-NEAREST NEIGHBORS (cv=5)
✅ Accuracy Scores: [0.75510204 0.7755102  0.75       0.77083333 0.79166667]
📊 Mean Accuracy: 0.7686
📉 Standard Deviation: 0.0149


In [26]:
mlp = MLPClassifier(hidden_layer_sizes=(100,),activation='relu',solver='sgd',early_stopping=True,
                    max_iter=500,random_state=42)
mlp.fit(X_train,y_train)
y_pred_mlp = mlp.predict(X_test)


print("\n🔷 MLPClassifier (Neural Network) ------------------")
accuracy(y_test,y_pred_mlp)
confusionMatrix(y_test,y_pred_mlp)
classificationReport(y_test,y_pred_mlp)
cross_validate_model(model,X_train,y_train,model_name="MLPClassifier")


🔷 MLPClassifier (Neural Network) ------------------
✅ Accuracy: 0.9016393442622951
📊 Confusion Matrix:
 [[25  4]
 [ 2 30]]
📝 Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.86      0.89        29
           1       0.88      0.94      0.91        32

    accuracy                           0.90        61
   macro avg       0.90      0.90      0.90        61
weighted avg       0.90      0.90      0.90        61


🔁 Cross-Validation for MLPCLASSIFIER (cv=5)
✅ Accuracy Scores: [0.75510204 0.7755102  0.75       0.77083333 0.79166667]
📊 Mean Accuracy: 0.7686
📉 Standard Deviation: 0.0149


In [28]:
dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train)
y_pred_tree =dtree.predict(X_test)

print("\n🔷 Decision Tree ------------------")
accuracy(y_test,y_pred_tree)
confusionMatrix(y_test,y_pred_tree)
classificationReport(y_test,y_pred_tree)
cross_validate_model(model,X_train,y_train,model_name="Decision Tree")



🔷 Decision Tree ------------------
✅ Accuracy: 0.8360655737704918
📊 Confusion Matrix:
 [[26  3]
 [ 7 25]]
📝 Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.90      0.84        29
           1       0.89      0.78      0.83        32

    accuracy                           0.84        61
   macro avg       0.84      0.84      0.84        61
weighted avg       0.84      0.84      0.84        61


🔁 Cross-Validation for DECISION TREE (cv=5)
✅ Accuracy Scores: [0.75510204 0.7755102  0.75       0.77083333 0.79166667]
📊 Mean Accuracy: 0.7686
📉 Standard Deviation: 0.0149


In [29]:
rf = RandomForestClassifier(n_estimators=100,max_depth=None,random_state=42)
rf.fit(X_train,y_train)
y_pred_rf = rf.predict(X_test)

print("\n🔷 Random Forest Classifier ------------------")
accuracy(y_test,y_pred_rf)
confusionMatrix(y_test,y_pred_rf)
classificationReport(y_test,y_pred_rf)
cross_validate_model(model,X_train,y_train,model_name="Random Forest")


🔷 Random Forest Classifier ------------------
✅ Accuracy: 0.8360655737704918
📊 Confusion Matrix:
 [[24  5]
 [ 5 27]]
📝 Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.83      0.83        29
           1       0.84      0.84      0.84        32

    accuracy                           0.84        61
   macro avg       0.84      0.84      0.84        61
weighted avg       0.84      0.84      0.84        61


🔁 Cross-Validation for RANDOM FOREST (cv=5)
✅ Accuracy Scores: [0.75510204 0.7755102  0.75       0.77083333 0.79166667]
📊 Mean Accuracy: 0.7686
📉 Standard Deviation: 0.0149


In [30]:
et = ExtraTreesClassifier(n_estimators=100,random_state=42)
et.fit(X_train,y_train)
y_pred_et=et.predict(X_test)

print("\n🔷 Extra Trees Classifier ------------------")
accuracy(y_test,y_pred_et)
confusionMatrix(y_test,y_pred_et)
classificationReport(y_test,y_pred_et)
cross_validate_model(model,X_train,y_train,model_name="Extra Trees")


🔷 Extra Trees Classifier ------------------
✅ Accuracy: 0.8524590163934426
📊 Confusion Matrix:
 [[25  4]
 [ 5 27]]
📝 Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.86      0.85        29
           1       0.87      0.84      0.86        32

    accuracy                           0.85        61
   macro avg       0.85      0.85      0.85        61
weighted avg       0.85      0.85      0.85        61


🔁 Cross-Validation for EXTRA TREES (cv=5)
✅ Accuracy Scores: [0.75510204 0.7755102  0.75       0.77083333 0.79166667]
📊 Mean Accuracy: 0.7686
📉 Standard Deviation: 0.0149


In [31]:
xgb_model = xgb(
    eval_metric='logloss',
    max_depth=4,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    n_estimators=100,
    random_state=42)
xgb_model.fit(X_train,y_train)
y_pred_xgb=xgb_model.predict(X_test)

print("\n🔷 XGBoost Classifier ------------------")
accuracy(y_test,y_pred_xgb)
confusionMatrix(y_test,y_pred_xgb)
classificationReport(y_test,y_pred_xgb)
cross_validate_model(model,X_train,y_train,model_name="XGBoost")


🔷 XGBoost Classifier ------------------
✅ Accuracy: 0.8524590163934426
📊 Confusion Matrix:
 [[25  4]
 [ 5 27]]
📝 Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.86      0.85        29
           1       0.87      0.84      0.86        32

    accuracy                           0.85        61
   macro avg       0.85      0.85      0.85        61
weighted avg       0.85      0.85      0.85        61


🔁 Cross-Validation for XGBOOST (cv=5)
✅ Accuracy Scores: [0.75510204 0.7755102  0.75       0.77083333 0.79166667]
📊 Mean Accuracy: 0.7686
📉 Standard Deviation: 0.0149


In [32]:
nb_model = GaussianNB()
nb_model.fit(X_train,y_train)
y_pred_nb = nb_model.predict(X_test)

print("\n🔷 Naive Bayes Classifier ------------------")
accuracy(y_test,y_pred_nb)
confusionMatrix(y_test,y_pred_nb)
classificationReport(y_test,y_pred_nb)
cross_validate_model(model,X_train,y_train,model_name="Naive Bayes")


🔷 Naive Bayes Classifier ------------------
✅ Accuracy: 0.8688524590163934
📊 Confusion Matrix:
 [[26  3]
 [ 5 27]]
📝 Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.90      0.87        29
           1       0.90      0.84      0.87        32

    accuracy                           0.87        61
   macro avg       0.87      0.87      0.87        61
weighted avg       0.87      0.87      0.87        61


🔁 Cross-Validation for NAIVE BAYES (cv=5)
✅ Accuracy Scores: [0.75510204 0.7755102  0.75       0.77083333 0.79166667]
📊 Mean Accuracy: 0.7686
📉 Standard Deviation: 0.0149


In [36]:


best_model = SVC(kernel='poly', probability=True)
best_model.fit(X_train, y_train)

import joblib
joblib.dump(best_model, 'model_svm.joblib')

print("Model trained and saved successfully!")


Model trained and saved successfully!
